In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder, OneHotEncoder
import pickle


In [59]:
data = pd.read_csv('Churn_Modelling.csv')

In [60]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [61]:
label_encoder_gender_2 = LabelEncoder()
data['Gender'] = label_encoder_gender_2.fit_transform(data['Gender'])


In [62]:
one_hot_encoder_geo_2 = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geo_2.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo_2.get_feature_names_out(['Geography']))

In [65]:
data = pd.concat([data.drop('Geography' , axis=1), geo_encoded_df], axis=1)

KeyError: "['Geography'] not found in axis"

In [66]:

data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [67]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [77]:
scaler2 = StandardScaler()
X_train = scaler2.fit_transform(X_train)
X_test = scaler2.transform(X_test)

In [78]:
with open('label_encoder_gender2.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender_2 , file)
with open('one_hot_encoder2.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo_2 , file)
with open('scaler2.pkl' , 'wb') as file:
    pickle.dump(scaler2 , file)

#ANN regression Problem Statement

In [79]:
#ANN regression Problem Statement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [80]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1)  # Output layer for regression
    ]
)

c:\Users\fk748\OneDrive\Desktop\ANN_Project_Implementation\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [81]:
#compile the model 
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [82]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [83]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime


#Set up TensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [84]:
#Set up Early Stopping 
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [85]:
#Train the Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 13386329088.0000 - mae: 100390.9062 - val_loss: 13015439360.0000 - val_mae: 98560.6016
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 13257466880.0000 - mae: 99742.4766 - val_loss: 12753174528.0000 - val_mae: 97233.7188
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 12777377792.0000 - mae: 97369.7109 - val_loss: 12055024640.0000 - val_mae: 93702.2266
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 11807454208.0000 - mae: 92577.4688 - val_loss: 10863785984.0000 - val_mae: 87682.5391
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 10371614720.0000 - mae: 85415.4219 - val_loss: 9288111104.0000 - val_mae: 79728.7031
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 8644843520.0000 - mae: 76813.0859 - val_loss: 7560197632.0000 - val_mae: 71074.8125
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6910622208.0000 - mae: 67999.2344 - val_loss: 5984155136.0

In [86]:
%load_ext tensorboard

In [88]:
%tensorboard --regressionlogs/fit

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
C:\Users\fk748\OneDrive\Desktop\ANN_Project_Implementation\venv\Lib\site-packages\tensorboard\default.py:30: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2026-01-04 23:23:36.806585: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-04 23:23:41.390028: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment v

In [89]:
#Evaluate the model on test data
test_loss  , test_mae = model.evaluate(X_test, y_test)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3359729664.0000 - mae: 50079.4141


In [90]:
test_mae

50079.4140625

In [91]:
model.save('regression_model.h5')